#Part I: Binary Classifier

In [ ]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 5.3 MB/s 
     |████████████████████████████████| 511.7 MB 6.3 kB/s 
     |████████████████████████████████| 438 kB 52.7 MB/s 
     |████████████████████████████████| 5.8 MB 40.9 MB/s 
     |████████████████████████████████| 1.6 MB 50.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.0 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 43.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import os, os.path
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
ROOT_PATH = "/content/drive/Shareddrives/CRP/"

In [ ]:
#load binary classifier model
y_reloaded_model = tf.keras.models.load_model((ROOT_PATH+'binary_model_350.h5'),custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
#Establish the directory of the folder with the files and the number of files the folder has
DIR = ROOT_PATH+'PerfCRP'
i=len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
j=i

In [ ]:
#Load each text file of the folder in a row in a dataframe
text = pd.read_csv(DIR+'/'+str(0)+'.txt', sep='\n', header=None,quoting=csv.QUOTE_NONE)[0].str.cat()
df = pd.DataFrame([text], columns=['text'])
for i in range(j):
  text = pd.read_csv(DIR+'/'+str(i)+'.txt', sep='\n', header=None,quoting=csv.QUOTE_NONE)[0].str.cat()
  dfs = pd.DataFrame([text], columns=['text'])
  df=df.append(dfs)
df.index = range(len(df))
df=df.drop(0)
df.index = range(len(df))

In [ ]:
# Apply model to dataframe/files
output=y_reloaded_model.predict(df)

2/2 [==============================] - 20s 5s/step


In [ ]:
#Drop those who have less than a 50% chance according to the model of being a biotech review
for i in range(j):
  if output[i][0]<0.50:
    df=df.drop(i)
df.index = range(len(df))

In [ ]:
#Final dataframe to use for other model
df

,text
0,"Biocatalysis expert Enzymaster (Germany, China..."
1,"Andera Partners, Paris based European private ..."
2,ransine Therapeutics Ltd has added £4.6m from ...
3,Minoryx Therapeutics SL has closed a €51m fina...
4,Avantium NV will get a €3m EU funding for a fe...
5,Cancer specialist Engimmune Therapeutics has r...
6,Italian start-up Enthera Pharmaceuticals has r...
7,Oxford-based Adaptimmune Therapeutics plc has ...
8,Oslo-based microbiome expert Bio-Me A/S has ra...
9,Swedish research-based late-stage biotech Cinc...


#Part II: NER

For the NER task, we use **BertForTokenClassification** which is included in the **Transformers library** by **HuggingFace**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
# input a list of news articles for NER
# suppose the name of the file is 'news.csv', with columns including 'Source', 'Title', 'Date', 'Content', 'URL'
# news_articles = pd.read_csv('news.csv')
# here we have the datafram df from binary classifier
news_articles = df

In [ ]:
MAX_LEN = 128
tokenizer = BertTokenizerFast.from_pretrained(ROOT_PATH+'ner_trained_model')
#ID - Label conversion
labels_to_ids = {'O': 0, 'B-s': 1, 'B-p': 2, 'B-i': 3, 'B-f': 4, 'B-y': 5, 'I-s': 6, 'I-p': 7, 'I-i': 8, 'I-f': 9, 'I-y': 10}
ids_to_labels = {0:'O', 1:'B-s', 2:'B-p', 3:'B-i', 4:'B-f', 5:'B-y', 6:'I-s', 7:'I-p', 8:'I-i', 9:'I-f', 10:'I-y'}

In [ ]:
model = BertForTokenClassification.from_pretrained(ROOT_PATH+'ner_trained_model', num_labels=len(labels_to_ids))
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
# function for returning list of recognized labels for each article
def inference_ner(news_article):
  sentence = news_article

  inputs = tokenizer(sentence.split(),
                      is_split_into_words=True, 
                      return_offsets_mapping=True, 
                      padding='max_length', 
                      truncation=True, 
                      max_length=MAX_LEN,
                      return_tensors="pt")

  # move to gpu
  ids = inputs["input_ids"].to(device)
  mask = inputs["attention_mask"].to(device)
  # forward pass
  outputs = model(ids, attention_mask=mask)
  logits = outputs[0]

  active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
  flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

  tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
  token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
  wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

  prediction = []
  for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
    #only predictions on first word pieces are important
    if mapping[0] == 0 and mapping[1] != 0:
      prediction.append(token_pred[1])
    else:
      continue
  return(prediction)

In [ ]:
# inference for all the input articles
# inference_result = pd.DataFrame(columns = ['Startup', 'Founder(s)', 'Investor(s)', 'City/Country', 'Year Founded', 'Source', 'Title', 'Date', 'Content', 'URL'])
inference_result = pd.DataFrame(columns = ['Startup', 'Founder(s)', 'Investor(s)', 'City/Country', 'Year Founded', 'News Content'])
for _, news_article in news_articles.iterrows(): 
  prediction = inference_ner(news_article['text'])
  news_article_split = news_article['text'].split()
  try:
    startup = set([news_article_split[i].strip().upper().translate(str.maketrans('', '', string.punctuation)) for i in [index for (index, item) in enumerate(prediction) if item == 'B-s' or item == 'I-s']])
    if len(startup) == 0:
      startup = np.NaN
  except:
    startup = np.NaN
  try:
    founder = set([news_article_split[i].strip().upper().translate(str.maketrans('', '', string.punctuation)) for i in [index for (index, item) in enumerate(prediction) if item == 'B-f' or item == 'I-f']])
    if len(founder) == 0:
      founder = np.NaN
  except:
    founder = np.NaN
  try:
    investor = set([news_article_split[i].strip().upper().translate(str.maketrans('', '', string.punctuation)) for i in [index for (index, item) in enumerate(prediction) if item == 'B-i' or item == 'I-i']])
    if len(investor) == 0:
      investor = np.NaN
  except:
    investor = np.NaN
  try:
    place = set([news_article_split[i].strip().upper().replace('BASED', '').replace('HEADQUARTRED', '').translate(str.maketrans('', '', string.punctuation)) for i in [index for (index, item) in enumerate(prediction) if item == 'B-p' or item == 'I-p']])
    if len(place) == 0:
      place = np.NaN   
  except:
    place = np.NaN
  try:
    year_founded = set([news_article_split[i].strip().upper().translate(str.maketrans('', '', string.punctuation)) for i in [index for (index, item) in enumerate(prediction) if item == 'B-y' or item == 'I-y']])
    if len(year_founded) == 0:
      year_founded = np.NaN
  except:
    year_founded = np.NaN
  inference_result = inference_result.append({
    'Startup': startup, 
    'Founder(s)': founder, 
    'Investor(s)': investor, 
    'City/Country': place, 
    'Year Founded': year_founded, 
    #'Source': news_article['Source'], 
    #'Title': news_article['Title'], 
    #'Date': news_article['Date'], 
    'News Content': news_article['text']#, 
    #'URL': news_article['URL']
    }, ignore_index=True)

In [ ]:
inference_result

,Startup,Founder(s),Investor(s),City/Country,Year Founded,News Content
0,{ENZYMASTER},NaN,"{BAYLAND, PUHUA, CAPITAL, CARBONNEUTRAL, YUANB...",NaN,NaN,"Biocatalysis expert Enzymaster (Germany, China..."
1,"{THERAPEUTICS, MINERALYS}",NaN,"{CAPITAL, ANDERA, MANAGEMENT, RA, PARTNERS}","{PARIS, PHILADELPHIA}",NaN,"Andera Partners, Paris based European private ..."
2,"{TRANSINE, THERAPEUTICS, RANSINE}",{THIRKETTLE},"{CAPITAL, DISCOVERY, FUND, VENTURES, EPIDAREX,...",NaN,NaN,ransine Therapeutics Ltd has added £4.6m from ...
3,"{MINORYX, THERAPEUTICS}",NaN,"{CAPITAL, CDTI, COLUMBUS, CAIXA, RISC, INNVIER...",{BARCELONA},NaN,Minoryx Therapeutics SL has closed a €51m fina...
4,"{WATERPROOF, AVANTIUM}",NaN,NaN,NaN,NaN,Avantium NV will get a €3m EU funding for a fe...
5,"{THERAPEUTICS, ENGIMMUNE}",NaN,"{BIOVENTURES, NOVO, AREA, HOLDINGS}",NaN,NaN,Cancer specialist Engimmune Therapeutics has r...
6,"{PHARMACEUTICALS, ENTHERA}","{CAMBONI, SPINELLI, SILVANO, GABRIELLA}","{INC, SOFINNOVA, ABBVIE, PARTNERS, T1D}",{ITALIAN},NaN,Italian start-up Enthera Pharmaceuticals has r...
7,"{THERAPEUTICS, ADAPTIMMUNE}",NaN,"{ENTERPRISE, CAPITAL, NEW, PORTFOLIO, MANAGEME...",{OXFORD},NaN,Oxford-based Adaptimmune Therapeutics plc has ...
8,"{BIOME, EXPERT}",NaN,NaN,{OSLO},NaN,Oslo-based microbiome expert Bio-Me A/S has ra...
9,"{CINCLUS, PHARMA}",NaN,"{AB, IRRUS, EIR, I, NOMINEE, APFONDEN, INVESTM...",NaN,NaN,Swedish research-based late-stage biotech Cinc...


In [ ]:
database = pd.read_csv(ROOT_PATH+'test_database.csv')
nrow_before = database.shape[0]
startup_list = []
for items in database['Name'].iteritems():
  startup_list.append(set(filter(None, [i.strip().upper().translate(str.maketrans('', '', string.punctuation)) for i in str(items[1]).split()])))

In [ ]:
# check if the company exists in the database
for _, i in inference_result.iterrows():
  if i['Startup'] not in startup_list:
    database.loc[len(database.index)] = [i['Startup'], i['Founder(s)'], i['City/Country'], i['Investor(s)'], i['Year Founded']]

In [ ]:
nrow_after = database.shape[0]
if nrow_after > nrow_before:
  database.to_csv(ROOT_PATH+'test_database.csv', index=False)
  print(str(nrow_after-nrow_before)+' startups have been added to the database!')
else:
  print('No startup is added to the database!')

No startup is added to the database!
